In [2]:
!pip -q install pandas numpy requests python-dateutil openpyxl

Define funciones de apoyo para perfilar datos, validar emails/teléfonos, parsear fechas, detectar columnas por patrón, etc.

In [4]:
import io, os, re, json, textwrap, datetime as dt
import numpy as np
import pandas as pd
import requests
from dateutil import parser

# Patrones para detectar tipos de columnas por nombre
DATE_LIKE_PAT  = re.compile(r"(date|fecha|created|updated|timestamp|time)", re.I)
EMAIL_LIKE_PAT = re.compile(r"(email|correo)", re.I)
PHONE_LIKE_PAT = re.compile(r"(phone|tel|mobile|móvil|cel)", re.I)
CITY_LIKE_PAT  = re.compile(r"(city|ciudad|municipio|billing\s*city|mailing\s*city)", re.I)
AMOUNT_LIKE_PAT = re.compile(r"(amount|monto|importe|value|price|revenue|total)", re.I)

# Perfilado rápido de columnas
def quick_profile(df: pd.DataFrame) -> pd.DataFrame:
    info = []
    for col in df.columns:
        nulls = df[col].isna().sum()
        nunique = df[col].nunique(dropna=True)
        sample_vals = df[col].dropna().astype(str).unique()[:5]
        info.append([col, str(df[col].dtype), df.shape[0], nulls, nunique, sample_vals])
    return pd.DataFrame(info, columns=["column","dtype","rows","nulls","nunique","sample"])

# Parsers / cleaners
def parse_maybe_date(s):
    try:
        return parser.parse(s)
    except Exception:
        return pd.NaT

def clean_phone(s):
    if pd.isna(s): return s
    digits = re.sub(r"\D", "", str(s))
    return digits if 7 <= len(digits) <= 15 else np.nan

def clean_email(s):
    if pd.isna(s): return s
    s = str(s).strip().lower()
    return s if re.match(r"^[^@]+@[^@]+\.[^@]+$", s) else np.nan

# Buscar primera columna que matchee un patrón
def find_first_matching_col(columns, pattern: re.Pattern, preferred_order=None):
    cols = list(columns)
    if preferred_order:
        for name in preferred_order:
            for c in cols:
                if c.strip().lower() == name.strip().lower():
                    return c
    for c in cols:
        if pattern.search(c):
            return c
    return None


In [7]:
fname_path = "/content/salesforce report.xlsx"

fname = os.path.basename(fname_path)

xls = pd.ExcelFile(fname_path)
sheet_to_use = None
for s in xls.sheet_names:
    tmp = pd.read_excel(fname_path, sheet_name=s)
    if tmp.shape[0] > 0 and tmp.shape[1] > 0:
        sheet_to_use = s
        break

if sheet_to_use is None:
    raise RuntimeError("No se encontró una hoja con datos.")

raw_df = pd.read_excel(fname_path, sheet_name=sheet_to_use)

print("Archivo:", fname)
print("Hoja seleccionada:", sheet_to_use)
print("Shape (raw):", raw_df.shape)
raw_df.head(3)



Archivo: salesforce report.xlsx
Hoja seleccionada: report1734540675541
Shape (raw): (2002, 19)


,Start Date,End Date,Campaign Record Type,Campaign categories,Campaign Name,Full Name,Primary Affiliation: Account Name,Email,Billing Country,Country Presence,Country,Country.1,Registered,Attended,Origin,Recibe newsletter,Social Cause,Campaign Subtype,#NAME?
0,2024-06-08 00:00:00,21/08/2024,Event,Tech; Fundraising,[Propel event] Aplica a grants con confianza (...,Valentina Medrano Coley,NaN,valentina.coley@cubosocial.org,NaN,NaN,NaN,NaN,0,0,Mafe's post,1,NaN,Propel event,NaN
1,2024-03-09 00:00:00,25/09/2024,Event,Tech,[Workshop] Fortalece tu historia de impacto (2...,Valentina Medrano Coley,NaN,valentina.coley@cubosocial.org,NaN,NaN,NaN,NaN,1,1,Mafe's post,1,NaN,Workshop,NaN
2,2024-06-08 00:00:00,21/08/2024,Event,Tech; Fundraising,[Propel event] Aplica a grants con confianza (...,Milagros Luque,Sembrando Juntos,mluque@sembrandojuntos.org,Perú,Perú,NaN,NaN,0,0,NaN,1,Educación,Propel event,NaN


In [8]:
profile_before = quick_profile(raw_df)
profile_before

,column,dtype,rows,nulls,nunique,sample
0,Start Date,object,2002,261,5,"[2024-06-08 00:00:00, 2024-03-09 00:00:00, 27/..."
1,End Date,object,2002,27,8,"[21/08/2024, 25/09/2024, 27/02/2024, 19/03/202..."
2,Campaign Record Type,object,2002,0,1,[Event]
3,Campaign categories,object,2002,371,2,"[Tech; Fundraising, Tech]"
4,Campaign Name,object,2002,0,9,[[Propel event] Aplica a grants con confianza ...
5,Full Name,object,2002,0,1227,"[Valentina Medrano Coley, Milagros Luque, Ánge..."
6,Primary Affiliation: Account Name,object,2002,249,727,"[Sembrando Juntos, Fundación Espacio Creativo,..."
7,Email,object,2002,0,1234,"[valentina.coley@cubosocial.org, mluque@sembra..."
8,Billing Country,object,2002,1702,21,"[Perú, Panamá, Colombia, Honduras, México]"
9,Country Presence,object,2002,1942,8,"[Perú, Panamá, Colombia, Chile, Mexico]"



*   Quita espacios y normaliza emails/ciudades
*   Valida emails/teléfonos
*   Parsea fechas
*   Convierte montos a numérico e imputa mediana cuando falten




In [10]:
df = raw_df.copy()

for c in df.columns:
    if df[c].dtype == object:
        df[c] = df[c].astype(str).str.strip()
        if EMAIL_LIKE_PAT.search(c):
            df[c] = df[c].map(clean_email)
        elif PHONE_LIKE_PAT.search(c):
            df[c] = df[c].map(clean_phone)
        elif CITY_LIKE_PAT.search(c):
            df[c] = df[c].str.title()

for c in df.columns:
    if DATE_LIKE_PAT.search(c):
        df[c] = df[c].apply(parse_maybe_date)

for c in df.columns:
    if df[c].dtype == object and AMOUNT_LIKE_PAT.search(c):
        df[c] = df[c].str.replace(",", "", regex=False)
        df[c] = pd.to_numeric(df[c], errors="coerce")

for c in df.columns:
    if pd.api.types.is_numeric_dtype(df[c]):
        if df[c].isna().mean() > 0 and df[c].notna().any():
            df[c] = df[c].fillna(df[c].median())


Si existe un Id (ContactId/AccountId/Record Id), se deduplica por ese campo; si no, por fila completa. Muestra cuántos registros se removieron.

In [11]:
id_like = find_first_matching_col(df.columns, re.compile(r"\b(id|record\s*id|contact\s*id|account\s*id)\b", re.I))
before_rows = df.shape[0]

if id_like:
    df = df.sort_values(by=[id_like]).drop_duplicates(subset=[id_like], keep="first").reset_index(drop=True)
else:
    df = df.drop_duplicates().reset_index(drop=True)

after_rows = df.shape[0]
print(f"Filas antes: {before_rows} | después de dedup: {after_rows} | removidas: {before_rows - after_rows}")


Filas antes: 2002 | después de dedup: 1997 | removidas: 5


Vuelve a perfilar para confirmar reducción de nulos/duplicados y tipo de datos resultante.


In [12]:
profile_after = quick_profile(df)
profile_after

,column,dtype,rows,nulls,nunique,sample
0,Start Date,datetime64[ns],1997,260,5,"[2024-06-08, 2024-03-09, 2024-11-27, 2024-09-3..."
1,End Date,datetime64[ns],1997,27,8,"[2024-08-21, 2024-09-25, 2024-02-27, 2024-03-1..."
2,Campaign Record Type,object,1997,0,1,[Event]
3,Campaign categories,object,1997,0,3,"[Tech; Fundraising, Tech, nan]"
4,Campaign Name,object,1997,0,9,[[Propel event] Aplica a grants con confianza ...
5,Full Name,object,1997,0,1227,"[Valentina Medrano Coley, Milagros Luque, Ánge..."
6,Primary Affiliation: Account Name,object,1997,0,728,"[nan, Sembrando Juntos, Fundación Espacio Crea..."
7,Email,object,1997,0,1234,"[valentina.coley@cubosocial.org, mluque@sembra..."
8,Billing Country,object,1997,0,22,"[nan, Perú, Panamá, Colombia, Honduras]"
9,Country Presence,object,1997,0,9,"[nan, Perú, Panamá, Colombia, Chile]"


Crea una carpeta que contiene una copia del excel original y el excel limpio.

In [13]:
OUTPUT_DIR = "outputs_part1"
os.makedirs(OUTPUT_DIR, exist_ok=True)

RAW_PATH   = os.path.join(OUTPUT_DIR, "dataset_raw.xlsx")
CLEAN_PATH = os.path.join(OUTPUT_DIR, "dataset_clean.csv")

with pd.ExcelWriter(RAW_PATH, engine="openpyxl") as writer:
    raw_df.to_excel(writer, sheet_name=str(sheet_to_use)[:31], index=False)

df.to_csv(CLEAN_PATH, index=False)
print("Guardados:")
print("•", RAW_PATH)
print("•", CLEAN_PATH)

Guardados:
• outputs_part1/dataset_raw.xlsx
• outputs_part1/dataset_clean.csv


Enriquecimiento con API pública:
Detecta City y Date; geocodifica y trae temperatura/precipitación diarias.

In [14]:
def find_date_col(columns):
    return find_first_matching_col(columns, DATE_LIKE_PAT, preferred_order=["Date","Created Date"])

def find_city_col(columns):
    return find_first_matching_col(columns, CITY_LIKE_PAT, preferred_order=["Mailing City","Billing City","City"])

def geocode_city(city: str):
    url = "https://geocoding-api.open-meteo.com/v1/search"
    r = requests.get(url, params={"name": city, "count": 1})
    if r.status_code != 200:
        return None
    js = r.json()
    if not js.get("results"):
        return None
    r0 = js["results"][0]
    return {"lat": r0["latitude"], "lon": r0["longitude"], "name": r0["name"], "country": r0.get("country_code")}

def get_daily_weather(lat, lon, start_date, end_date):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_mean,precipitation_sum",
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "UTC"
    }
    r = requests.get(url, params=params)
    if r.status_code != 200:
        return pd.DataFrame()
    js = r.json()
    if "daily" not in js:
        return pd.DataFrame()
    d = js["daily"]
    out = pd.DataFrame({
        "date": pd.to_datetime(d["time"]),
        "temp_mean_c": d.get("temperature_2m_mean", [np.nan]*len(d["time"])),
        "precip_mm": d.get("precipitation_sum", [np.nan]*len(d["time"]))
    })
    return out

date_col = find_date_col(df.columns)
city_col = find_city_col(df.columns)
print("date_col:", date_col, "| city_col:", city_col)

enriched_df = df.copy()
weather_frames = []

if city_col and date_col:
    dmin = pd.to_datetime(enriched_df[date_col]).min().date()
    dmax = pd.to_datetime(enriched_df[date_col]).max().date()
    print("Date range:", dmin, "->", dmax)

    unique_cities = enriched_df[city_col].dropna().unique().tolist()[:20]
    for city in unique_cities:
        geo = geocode_city(city)
        if not geo:
            print("Sin geocódigo:", city);
            continue
        weather = get_daily_weather(geo["lat"], geo["lon"], str(dmin), str(dmax))
        if weather.empty:
            print("Sin clima:", city)
            continue
        weather[city_col] = city
        weather_frames.append(weather)

    if weather_frames:
        weather_all = pd.concat(weather_frames, ignore_index=True)
        enriched_df["_merge_date"] = pd.to_datetime(enriched_df[date_col]).dt.date
        weather_all["_merge_date"] = pd.to_datetime(weather_all["date"]).dt.date
        enriched_df = enriched_df.merge(
            weather_all[[city_col, "_merge_date", "temp_mean_c", "precip_mm"]],
            how="left",
            on=[city_col, "_merge_date"]
        ).drop(columns=["_merge_date"])
else:
    print("ℹNo se detectó City/Date; omitiendo enriquecimiento.")

ENRICHED_PATH = os.path.join(OUTPUT_DIR, "dataset_enriched.csv")
enriched_df.to_csv(ENRICHED_PATH, index=False)
print("Guardado enriquecido:", ENRICHED_PATH)
enriched_df.head(3)


date_col: Start Date | city_col: None
ℹNo se detectó City/Date; omitiendo enriquecimiento.
Guardado enriquecido: outputs_part1/dataset_enriched.csv


,Start Date,End Date,Campaign Record Type,Campaign categories,Campaign Name,Full Name,Primary Affiliation: Account Name,Email,Billing Country,Country Presence,Country,Country.1,Registered,Attended,Origin,Recibe newsletter,Social Cause,Campaign Subtype,#NAME?
0,2024-06-08,2024-08-21,Event,Tech; Fundraising,[Propel event] Aplica a grants con confianza (...,Valentina Medrano Coley,nan,valentina.coley@cubosocial.org,nan,nan,nan,nan,0,0,Mafe's post,1,nan,Propel event,NaN
1,2024-03-09,2024-09-25,Event,Tech,[Workshop] Fortalece tu historia de impacto (2...,Valentina Medrano Coley,nan,valentina.coley@cubosocial.org,nan,nan,nan,nan,1,1,Mafe's post,1,nan,Workshop,NaN
2,2024-06-08,2024-08-21,Event,Tech; Fundraising,[Propel event] Aplica a grants con confianza (...,Milagros Luque,Sembrando Juntos,mluque@sembrandojuntos.org,Perú,Perú,nan,nan,0,0,nan,1,Educación,Propel event,NaN


Esta última celda genera el reporte, de igual forma lo enviaré en pdf.

In [15]:
issues = []
null_summary = raw_df.isna().sum()
for c, v in null_summary.items():
    if v > 0:
        issues.append(f"- Missing values in **{c}**: {v} nulls")

bad_emails = 0
bad_phones = 0
for c in raw_df.columns:
    if EMAIL_LIKE_PAT.search(c):
        bad_emails += (raw_df[c].dropna().astype(str).str.match(r"^[^@]+@[^@]+\.[^@]+$") == False).sum()
    if PHONE_LIKE_PAT.search(c):
        bad_phones += (raw_df[c].dropna().astype(str).str.replace(r"\D", "", regex=True).str.len().between(7,15) == False).sum()

if bad_emails:
    issues.append(f"- Malformed emails: ~{bad_emails}")
if bad_phones:
    issues.append(f"- Malformed phones: ~{bad_phones}")
if before_rows != after_rows:
    issues.append(f"- Duplicates removed: {before_rows - after_rows}")

if not issues:
    issues.append("- No major quality issues detected beyond standardization.")

report_md = f"""# Job Sample – Part 1 Report (Data Cleaning & API Integration)

**File:** `{fname}` (sheet: `{sheet_to_use}`)
**Rows/Cols (raw):** {raw_df.shape[0]} / {raw_df.shape[1]}
**Rows after cleaning:** {after_rows}

## Issues Found
{os.linesep.join(issues)}

## Cleaning Steps
- Whitespace trimming; text normalization (emails lower, cities Title Case).
- Email/phone validation & standardization.
- Date parsing for date-like columns.
- Numeric coercion for amount-like fields; median imputation when partially missing.
- Deduplication ({'by ' + find_first_matching_col(df.columns, re.compile(r"\\b(id|record\\s*id|contact\\s*id|account\\s*id)\\b", re.I)) if find_first_matching_col(df.columns, re.compile(r"\\b(id|record\\s*id|contact\\s*id|account\\s*id)\\b", re.I)) else 'full-row'}) removing {before_rows - after_rows} rows.

## Proposed Data Structuring / CRM (Salesforce)
- **Account**(AccountId PK, Name, Industry, BillingCountry, CreatedDate)
- **Contact**(ContactId PK, AccountId FK, FirstName, LastName, Email(unique), Phone(E.164))
- **Lead**(LeadId PK, Company, Email, Phone, LeadSource, Status)
- **Interaction__c**(InteractionId PK, AccountId/ContactId FK, InteractionDate, Type, Amount__c, SourceSystem__c, CreatedTS__c)
- **Location__c**(LocationId PK, City, Country, Latitude, Longitude)
- **WeatherDaily__c**(WeatherId PK, LocationId FK, Date, TempMeanC__c, PrecipMM__c, Source="Open-Meteo")

**Validations:** required fields, email regex, phone length, historical dates not in the future.
**ETL/Sync:** scheduled jobs to refresh enrichments; maintain lineage (`SourceSystem`, `IngestedAt`).

## API Enrichment
If available, joined by (City, Date) to daily mean temperature/precipitation from Open-Meteo.
"""

REPORT_PATH = os.path.join(OUTPUT_DIR, "Part1_Report.md")
with open(REPORT_PATH, "w", encoding="utf-8") as f:
    f.write(report_md)

print("Reporte listo:", REPORT_PATH)
issues = []
null_summary = raw_df.isna().sum()
for c, v in null_summary.items():
    if v > 0:
        issues.append(f"- Missing values in **{c}**: {v} nulls")

bad_emails = 0
bad_phones = 0
for c in raw_df.columns:
    if EMAIL_LIKE_PAT.search(c):
        bad_emails += (raw_df[c].dropna().astype(str).str.match(r"^[^@]+@[^@]+\.[^@]+$") == False).sum()
    if PHONE_LIKE_PAT.search(c):
        bad_phones += (raw_df[c].dropna().astype(str).str.replace(r"\D", "", regex=True).str.len().between(7,15) == False).sum()

if bad_emails:
    issues.append(f"- Malformed emails: ~{bad_emails}")
if bad_phones:
    issues.append(f"- Malformed phones: ~{bad_phones}")
if before_rows != after_rows:
    issues.append(f"- Duplicates removed: {before_rows - after_rows}")

if not issues:
    issues.append("- No major quality issues detected beyond standardization.")

report_md = f"""# Job Sample – Part 1 Report (Data Cleaning & API Integration)

**File:** `{fname}` (sheet: `{sheet_to_use}`)
**Rows/Cols (raw):** {raw_df.shape[0]} / {raw_df.shape[1]}
**Rows after cleaning:** {after_rows}

## Issues Found
{os.linesep.join(issues)}

## Cleaning Steps
- Whitespace trimming; text normalization (emails lower, cities Title Case).
- Email/phone validation & standardization.
- Date parsing for date-like columns.
- Numeric coercion for amount-like fields; median imputation when partially missing.
- Deduplication ({'by ' + find_first_matching_col(df.columns, re.compile(r"\\b(id|record\\s*id|contact\\s*id|account\\s*id)\\b", re.I)) if find_first_matching_col(df.columns, re.compile(r"\\b(id|record\\s*id|contact\\s*id|account\\s*id)\\b", re.I)) else 'full-row'}) removing {before_rows - after_rows} rows.

## Proposed Data Structuring / CRM (Salesforce)
- **Account**(AccountId PK, Name, Industry, BillingCountry, CreatedDate)
- **Contact**(ContactId PK, AccountId FK, FirstName, LastName, Email(unique), Phone(E.164))
- **Lead**(LeadId PK, Company, Email, Phone, LeadSource, Status)
- **Interaction__c**(InteractionId PK, AccountId/ContactId FK, InteractionDate, Type, Amount__c, SourceSystem__c, CreatedTS__c)
- **Location__c**(LocationId PK, City, Country, Latitude, Longitude)
- **WeatherDaily__c**(WeatherId PK, LocationId FK, Date, TempMeanC__c, PrecipMM__c, Source="Open-Meteo")

**Validations:** required fields, email regex, phone length, historical dates not in the future.
**ETL/Sync:** scheduled jobs to refresh enrichments; maintain lineage (`SourceSystem`, `IngestedAt`).

## API Enrichment
If available, joined by (City, Date) to daily mean temperature/precipitation from Open-Meteo.
"""

REPORT_PATH = os.path.join(OUTPUT_DIR, "Part1_Report.md")
with open(REPORT_PATH, "w", encoding="utf-8") as f:
    f.write(report_md)

print("Reporte listo:", REPORT_PATH)


Reporte listo: outputs_part1/Part1_Report.md
Reporte listo: outputs_part1/Part1_Report.md
